# Modelación del comportamiento de los clientes de Megaline

## Tabla de contenidos

* [Introducción](#intro) 
* [Objetivos](#objective)
* [Revisión de los datos preprocesados](#data_review)
* [Segmentación de datos](#data_segmentation)
* [Evaluación de modelos](#model_evaluation)
* [Validación del modelo seleccionado](#model_validation)
* [Conclusiones](#end)

## Introducción<a id='intro'></a>

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

## Objetivo<a id='objective'></a>

* Crear un modelo de clasificación, con exactitud del al menos 0.75, para asesorar a los clientes en su selección de planes de consumo 
 

## Revisión de los datos preprocesados<a id='data_review'></a>

In [3]:
import pandas as pd
users_behaviour = pd.read_csv("/datasets/users_behavior.csv")

In [4]:
users_behaviour.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
users_behaviour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Efectivamente, los datos se encuentran limpios y listos para la modelación. 

## Segmentación de datos<a id='data_segmentation'></a>

La segmentación de datos se realizará en dos etapas para obtener los datos de entrenamiento (60 %), prueba (20 %) y validación (20%).

In [91]:
# División de datos en conjuntos de entrenamiento y prueba

from sklearn.model_selection import train_test_split
users_behaviour_train, users_behaviour_test = train_test_split(users_behaviour, test_size=0.20, random_state=12345)

In [82]:
# Segmentación de características y de datos objetivo

features_train = users_behaviour_train.drop(['is_ultra'], axis=1)
features_test = users_behaviour_valid.drop(['is_ultra'], axis=1)
target_train = users_behaviour_train['is_ultra']
target_test = users_behaviour_valid['is_ultra']

## Evaluación de modelos<a id='model_evaluation'></a>

Este es un problema de clasificación. Por lo tanto, se evaluarán tres modelos. 1) árbol de decisión, 2) bosque aleatorio, y 3) regresión logística.

In [83]:
# Evaluación del modelo de árbol de decisión con datos de entrenamiento

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

best_model = None
best_result = 0

for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # crea un modelo con la profundidad proporcionada
	model.fit(features_train,target_train) # entrena el modelo 
	predictions = model.predict(features_train) #hace predicciones con daos de entrenamiento
	result = accuracy_score(target_train,predictions) # calcula la exactitud
    
	if result > best_result:
		best_model = model
		best_result = result
            
print("Exactitud del mejor modelo en el conjunto de entrenamiento:", best_result)

Exactitud del mejor modelo en el conjunto de entrenamiento: 0.8144690781796966


In [86]:
# Evaluación del modelo de árbol de decisión con datos de prueba

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

best_model = None
best_result = 0
for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # crea un modelo con la profundidad proporcionada
	model.fit(features_train,target_train) # entrena el modelo 
	predictions = model.predict(features_test) # obtiene las predicciones del modelo
	result = accuracy_score(target_test,predictions) # calcula la exactitud
    
	if result > best_result:
		best_model = model
		best_result = result
            
print("Exactitud del mejor modelo en el conjunto de prueba:", best_result)

Exactitud del mejor modelo en el conjunto de prueba: 0.7898009950248757


La evaluación del modelo de árbol de desición determina que existe un ligero sobreajuste (aprox + 2 %). 

In [87]:
# Evaluación del modelo de bosque aleatorio con datos de entrenamiento

from sklearn.ensemble import RandomForestClassifier

best_score = 0
best_est = 0

for est in range(1,11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # configura el número de árboles
    model.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(features_train,target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
    if score > best_score:
        best_score = score #guarda la mejor puntuación de accuracy en el conjunto de entrenamiento
        best_est = est# guarda el número de estimadores que corresponden a la mejor puntuación de accuracy

print("Accuracy del mejor modelo en el conjunto de entrenamiento (n_estimators = {}): {}".format(best_est, best_score))

Accuracy del mejor modelo en el conjunto de entrenamiento (n_estimators = 9): 0.9782185919875535


In [88]:
# Evaluación del modelo de bosque aleatorio con datos de prueba

from sklearn.ensemble import RandomForestClassifier

best_score = 0
best_est = 0

for est in range(1,11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # configura el número de árboles
    model.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(features_test,target_test) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score #guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est# guarda el número de estimadores que corresponden a la mejor puntuación de accuracy

print("Accuracy del mejor modelo en el conjunto de prueba (n_estimators = {}): {}".format(best_est, best_score))

Accuracy del mejor modelo en el conjunto de prueba (n_estimators = 10): 0.8121890547263682


La evaluación del modelo de bosque aleatorio determina que también existe sobreajuste considerable (aprox. + 16 %). 

In [90]:
# Evaluación del modelo de regresión logística

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=12345, solver='liblinear')# inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features_train,target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_test = model.score(features_test,target_test) # calcula la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de prueba:", score_test)


Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7016725009723843
Accuracy del modelo de regresión logística en el conjunto de prueba: 0.7052238805970149


El modelo de regresión logística no presenta sobreajuste. No obstante, al encontrarse por debajo del umbral de 0.75, el modelo se encuentra subajustado (aprox. - 5 %). 

## Validación del modelo seleccionado<a id='model_validation'></a>

De la evaluación de modelos, se concluye que el modelo que mejores resultados brinda es el de árbol de decisión. Alcanza una exactitud mayor a 0.75 en el conjunto de prueba (0.79), y por otra parte, es el que mejor balance presenta con respecto al ajuste (sobreajuste de + 2%). En esta sección, se validará el modelo.

In [95]:
# División de datos en conjuntos de entrenamiento y validación

from sklearn.model_selection import train_test_split
users_behaviour_train, users_behaviour_valid = train_test_split(users_behaviour, test_size=0.25, random_state=12345)

In [96]:
# Segmentación de características y de datos objetivo

features_train = users_behaviour_train.drop(['is_ultra'], axis=1)
features_valid = users_behaviour_valid.drop(['is_ultra'], axis=1)
target_train = users_behaviour_train['is_ultra']
target_valid = users_behaviour_valid['is_ultra']

In [97]:
# Validación del modelo de árbol de decisión con datos de entrenamiento

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

best_model = None
best_result = 0

for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # crea un modelo con la profundidad proporcionada
	model.fit(features_train,target_train) # entrena el modelo 
	predictions = model.predict(features_train) #hace predicciones con daos de entrenamiento
	result = accuracy_score(target_train,predictions) # calcula la exactitud
    
	if result > best_result:
		best_model = model
		best_result = result
            
print("Exactitud del mejor modelo en el conjunto de entrenamiento:", best_result)

Exactitud del mejor modelo en el conjunto de entrenamiento: 0.8149377593360996


In [98]:
# Evaluación del modelo de árbol de decisión con datos de validación

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

best_model = None
best_result = 0
for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # crea un modelo con la profundidad proporcionada
	model.fit(features_train,target_train) # entrena el modelo 
	predictions = model.predict(features_valid) # obtiene las predicciones del modelo
	result = accuracy_score(target_valid,predictions) # calcula la exactitud
    
	if result > best_result:
		best_model = model
		best_result = result
            
print("Exactitud del mejor modelo en el conjunto de validación:", best_result)

Exactitud del mejor modelo en el conjunto de validación: 0.7885572139303483


Con datos de validación, el modelo de árbol de decisión presenta resultados similares a los usados con los datos de prueba. Por lo tanto, el modelo predictivo esta listo para ser puesto en producción. 

Por otra parte, en lo que respecta al análisis de cordura, al ser este un problema de clasificación binario, el porcentaje de acertar en la predicción al azar es del 50 %. En virud de que el modelo validado tiene una exactitud del 79 %, el modelo tiene sentido y puede ser útil.

## Conclusiones<a id='end'></a>

1. De la evaluación de modelos, se concluye que el modelo que mejores resultados brinda es el de árbol de decisión. Alcanza una exactitud mayor a 0.75 en el conjunto de prueba (0.79), y por otra parte, es el que mejor balance presenta con respecto al ajuste (sobreajuste de + 2%).
2. Al ser un problema de clasificación binario, el porcentaje de acertar en la predicción al azar es del 50 %. En virtud de que el modelo validado tiene una exactitud del 79 %, el modelo tiene sentido y puede ser útil.
3. Con datos de validación, el modelo de árbol de decisión presenta resultados similares a los usados con los datos de prueba. Por lo tanto, el modelo predictivo esta listo para ser puesto en producción.